In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [8]:
# 1. Selenium으로 데이터 크롤링
def scrape_kbo_rankings():
    # 크롬 드라이버 초기화
    driver = webdriver.Chrome()
    driver.get("https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx")
    time.sleep(3)
    
    # 팀 데이터를 저장할 딕셔너리
    team_data = {}
    
    # 해당 테이블의 summary 값 (구분 기준)
    target_summary = "순위, 팀명,승,패,무,승률,승차,최근10경기,연속,홈,방문"

    # 필요한 날짜 범위만큼 반복 (예: 최근 5일 데이터)
    for _ in range(5):
        # print(f"\n=== {day + 1}번째 날짜 크롤링 시작 ===")  # 디버깅용 메시지
        # 날짜 추출
        date_element = driver.find_element(By.ID, "cphContents_cphContents_cphContents_lblSearchDateTitle")
        current_date = date_element.text.strip()
        # print(f"현재 날짜: {current_date}")  # 가져온 날짜 출력 (디버깅용)
        
        # summary 속성을 기준으로 테이블 선택
        table = None
        tables = driver.find_elements(By.TAG_NAME, "table")
        for tbl in tables:
            summary_attribute = tbl.get_attribute("summary")
            if summary_attribute == target_summary:
                table = tbl
                break
            
        # 테이블을 찾지 못한 경우
        if table is None:
            # print(f"'{target_summary}'를 가진 테이블을 찾지 못했습니다. 다음 날짜로 이동합니다.")
            continue
        
        
         # 테이블 행 데이터 가져오기
        rows = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')
        # print(f"총 {len(rows)}개의 행이 발견되었습니다! (summary='{target_summary}')")

        for i, row in enumerate(rows):
            cols = row.find_elements(By.TAG_NAME, 'td')
            
            # 디버깅: 각 행의 모든 열 데이터 출력
            # print(f"행 {i + 1} 데이터: {[col.text.strip() for col in cols]}")

            # 데이터 행 필터링 (열이 부족하거나, 첫 번째 열이 숫자 형식이 아니면 스킵)
            if len(cols) < 2:
                # print(f"데이터가 부족한 행입니다. 건너뜁니다.")
                continue

            rank_text = cols[0].text.strip()  # 첫 번째 칸: 순위
            if not rank_text.isdigit():  # 숫자가 아니면 무시
                # print(f"순위 값이 숫자가 아닙니다: {rank_text}")
                continue

            # 정제된 데이터 추출
            rank = int(rank_text)
            team_name = cols[1].text.strip()  # 두 번째 칸: 팀 이름
            # print(f"  - 순위 {rank}: {team_name}")

            # 팀 데이터 저장
            if team_name not in team_data:
                team_data[team_name] = []
            
            team_data[team_name].append({"date": current_date, "rank": rank})

        # "이전 날짜" 버튼 클릭
        prev_date_button = driver.find_element(By.ID, "cphContents_cphContents_cphContents_btnPreDate")
        prev_date_button.click()
        # print("이전 날짜로 이동 중...")
        time.sleep(3)  # 데이터 로드 대기

    driver.quit()
    # print("\n크롤링 완료! 팀별 데이터를 딕셔너리에 저장했습니다.")
    
    # print("\n=== 수집된 데이터 일부 확인 ===")
    # for team, records in team_data.items():
    #     print(f"팀: {team}, 데이터: {records[:2]}")  # 각 팀의 첫 2개 데이터만 출력

    return team_data

In [9]:
# scrape_kbo_rankings 함수를 실행하고, 결과를 변수에 저장
team_data = scrape_kbo_rankings()

# # 결과를 확인
# print(team_data)

In [10]:
import json

# JSON 파일 저장 함수 정의
def save_to_json(data, filename="kbo_rankings.json"):
    """
    딕셔너리를 JSON 파일로 저장하는 함수
    :param data: 저장할 데이터 (딕셔너리)
    :param filename: 생성할 JSON 파일 이름 (기본값: "kbo_rankings.json")
    """
    try:
        with open(filename, "w", encoding="utf-8") as json_file:
            json.dump(data, json_file, indent=4, ensure_ascii=False)  # JSON 파일 생성
        print(f"\n데이터가 JSON 파일 '{filename}'로 저장되었습니다.")
    except Exception as e:
        print(f"\nJSON 파일 저장 중 오류 발생: {e}")

In [ ]:
save_to_json(team_data)